In [2]:
import os
import sys
import json
import pickle
import pandas as pd

In [3]:
# list all the files in the directory
directory = "../data/libraries/BERKELEY-LAB/nf_output/fragmentationtrees/"
files = os.listdir(directory)
print (len(files))

7968


In [4]:
with open(os.path.join("../data/libraries/BERKELEY-LAB/data_dict_filtered.pkl"), "rb") as f:
    data_dict_filtered = pickle.load(f)

In [14]:
# get one instance of the dictionary
print (data_dict_filtered[files[0].split('_')[0]])

{'spectrum_id': 'CCMSLIB00010112368', 'source_file': 'tharwood/spectral_libraries/BERKELEY-LAB.mgf', 'task': '3dacd5cf9c8d4c3d8596c71bf3234fbd', 'scan': '9934', 'ms_level': '2', 'library_membership': 'BERKELEY-LAB', 'spectrum_status': '1', 'peaks_json': '[[82.389229,4067.000000],[97.653358,3396.000000],[98.701729,3064.000000],[104.272476,4583.000000],[118.642662,3786.000000],[127.181679,3824.000000],[132.433853,3219.000000],[157.955811,3995.000000],[212.081497,76441.000000],[213.090424,34772.000000],[216.058289,5102.000000],[230.038010,60683.000000],[244.053558,590029.000000],[256.041077,4010.000000],[257.048615,173171.000000],[262.063873,44844.000000],[271.064362,1760259.000000],[299.059265,5541474.000000],[354.633240,4181.000000],[442.095490,13629.000000],[460.104309,1523141.000000]]', 'splash': 'null-null-null-null', 'submit_user': 'mwang87', 'Compound_Name': '2-[9-(4-methylphenyl)-6-oxohydropurin-8-ylthio]-N-[3-(trifluoromethyl)phenyl]a cetamide CollisionEnergy:102040', 'Ion_Source

In [42]:
def binary_search_fragments(fragments, search_mz, threshold):
    # Binary search implementation
    left = 0
    right = len(fragments)

    while left < right:
        mid = (left + right) // 2
        mz = fragments[mid]['mz']
        # print (left, right, mid, mz, search_mz)
        diff = abs(mz - search_mz)
        if right - left == 1:
            if diff <= threshold:
                return left
            else:
                return -1
        elif mz >= search_mz:
            left = mid
        else:
            right = mid

    return -1  # Return -1 if fragment not found

In [55]:
min = 100000
max = 0
average = 0
df = pd.DataFrame(columns=['file', 'fragment_count', 'score', 'peak_found'])
for file in files:
    id = file.split('_')[0]
    try:
        peaks = json.loads(data_dict_filtered[id]['peaks_json'])
    except:
        continue
    data = json.load(open(os.path.join(directory, file)))
    count = 0
    for fragment in data['fragments']:
        count += 1
        if len(fragment['peaks']) > 1:
            print (file, fragment)
        score = fragment['score']

    peakFound = 0
    for peak in peaks:
        if binary_search_fragments(data['fragments'], peak[0], 0.0000001) != -1:
            peakFound += 1
    if peakFound == 0:
        print (file, id)


    df = pd.concat([df, pd.DataFrame([[id, float(count), score, float(peakFound)]], columns=['file', 'fragment_count', 'score', 'peak_found'])])

In [41]:
binary_search_fragments(data['fragments'], 417.11734, 0.1)

0 9 4 359.075348 417.11734
0 4 2 375.106842 417.11734
0 2 1 417.11734 417.11734
1 2 1 417.11734 417.11734


1

In [49]:
id
data = json.load(open(os.path.join(directory, file)))
peaks = json.loads(data_dict_filtered[id]['peaks_json'])
# print in reverse order
peaks.sort(key=lambda x: x[0], reverse=True)
print ([fragment['mz'] for fragment in data['fragments']])
peakFound
for peak in peaks:
    if binary_search_fragments(data['fragments'], peak[0], 0.1) != -1:
        peakFound += 1
    print (peak[0], binary_search_fragments(data['fragments'], peak[0], 0.1))
print (peakFound)

[459.127625, 417.11734, 375.106842, 360.083313, 359.075348, 345.059906, 342.072845, 317.065125, 314.07843]
459.127625 0
419.326996 -1
417.11734 1
416.107269 -1
414.921661 -1
402.093384 -1
375.106842 2
374.094604 -1
373.233398 -1
373.094147 -1
360.083313 3
359.075348 4
357.099091 -1
345.059906 5
343.079651 -1
342.072845 6
341.06485 -1
331.07843 -1
329.066162 -1
327.049286 -1
325.07019 -1
324.061798 -1
317.105286 -1
317.065125 7
315.088104 -1
314.07843 8
313.070831 -1
312.061646 -1
311.054596 -1
310.045685 -1
299.053345 -1
296.068939 -1
285.075012 -1
274.440582 -1
151.038925 -1
143.204956 -1
118.648514 -1
92.37162 -1
82.393227 -1
73.948357 -1
11


In [56]:
# print stats about dataframe
print (df.describe(include='all'))

                      file  fragment_count        score   peak_found
count                 4101     4101.000000  4101.000000  4101.000000
unique                4101             NaN          NaN          NaN
top     CCMSLIB00010112368             NaN          NaN          NaN
freq                     1             NaN          NaN          NaN
mean                   NaN       18.381370     3.065446    18.290173
std                    NaN       14.898482     2.914729    14.923845
min                    NaN        1.000000    -0.615768     1.000000
25%                    NaN        7.000000     1.344323     7.000000
50%                    NaN       15.000000     2.213368    14.000000
75%                    NaN       24.000000     3.757105    24.000000
max                    NaN       60.000000    18.612715    60.000000
